## Using Music21 Library

In [1]:
song_name = 'BlueStone_LastDungeon.mid'

In [4]:
import numpy as np
from music21 import *
import tensorflow as tf
import glob
import keras
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import os
import pickle
from keras.models import load_model
#from keras.models import model_from_json

In [5]:
notes = []
for file in glob.glob('C:/Users/singh/Downloads/Music Generation/midi_songs/classical-piano-midi/' + song_name):
    midi = converter.parse(file)
    notes_to_parse = None
    
    notes_to_parse = midi.flat.notes
    
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
print(len(notes))

682


In [6]:
seq_length = 100

pitchnames = sorted(set(notes))

n_vocab = len(pitchnames)

notes_to_int = dict((note, n) for n, note in enumerate(pitchnames))

input_seq = []
output_seq = []

for i in range(len(notes) - seq_length):
    cur_seq = notes[i:(i+seq_length)]
    out_notes = notes[(i+seq_length)]
    input_seq.append([notes_to_int[char] for char in cur_seq])
    output_seq.append(notes_to_int[out_notes])
    
total_inputs = len(input_seq)
input_seq = np.reshape(input_seq, (total_inputs, seq_length, 1))

input_seq = input_seq/float(n_vocab)
print(input_seq.shape)
output_seq = np_utils.to_categorical(output_seq)
print(output_seq.shape)

(582, 100, 1)
(582, 65)


## Using 1 LSTM layer

In [28]:
model_1 = tf.keras.Sequential([
    tf.keras.layers.LSTM(256, input_shape = (input_seq.shape[1], input_seq.shape[2]), recurrent_dropout = 0.3,),
    tf.keras.layers.Dense(n_vocab, activation  = 'softmax')
])
model_1.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [29]:
#filepath="C:/Users/singh/Downloads/Music Generation/Checkpoints/Music21/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
filepath="C:/Users/singh/Downloads/Music Generation/Checkpoints/Music21/weights.best.hdf5"

checkpoint = ModelCheckpoint(filepath,
                             monitor='loss', 
                             verbose=0, 
                             save_best_only=True, 
                             mode='min')
callbacks_list = [checkpoint]

model_1.fit(input_seq, output_seq, epochs=200,  batch_size=128, callbacks = callbacks_list)

Train on 582 samples
Epoch 1/200
582/582 [==============================] - 4s 7ms/sample - loss: 4.1134
Epoch 2/200
582/582 [==============================] - 3s 5ms/sample - loss: 3.6408
Epoch 3/200
582/582 [==============================] - 3s 5ms/sample - loss: 3.4974
Epoch 4/200
582/582 [==============================] - 3s 5ms/sample - loss: 3.4732
Epoch 5/200
582/582 [==============================] - 3s 5ms/sample - loss: 3.4439
Epoch 6/200
582/582 [==============================] - 3s 5ms/sample - loss: 3.4474
Epoch 7/200
582/582 [==============================] - 3s 5ms/sample - loss: 3.4403
Epoch 8/200
582/582 [==============================] - 3s 5ms/sample - loss: 3.4371
Epoch 9/200
582/582 [==============================] - 3s 5ms/sample - loss: 3.4341
Epoch 10/200
582/582 [==============================] - 3s 5ms/sample - loss: 3.4320
Epoch 11/200
582/582 [==============================] - 3s 5ms/sample - loss: 3.4326
Epoch 12/200
582/582 [==============================]

582/582 [==============================] - 2s 4ms/sample - loss: 1.2867
Epoch 193/200
582/582 [==============================] - 2s 4ms/sample - loss: 1.2928
Epoch 194/200
582/582 [==============================] - 3s 4ms/sample - loss: 1.2434
Epoch 195/200
582/582 [==============================] - 3s 5ms/sample - loss: 1.2613
Epoch 196/200
582/582 [==============================] - 3s 4ms/sample - loss: 1.2429
Epoch 197/200
582/582 [==============================] - 3s 4ms/sample - loss: 1.2208
Epoch 198/200
582/582 [==============================] - 2s 4ms/sample - loss: 1.2354
Epoch 199/200
582/582 [==============================] - 2s 4ms/sample - loss: 1.2440
Epoch 200/200
582/582 [==============================] - 2s 4ms/sample - loss: 1.2309


In [32]:
model_1.save('music21_model_1.h5')  # creates a HDF5 file 'my_model.h5'
del model_1

model_1 = tf.keras.models.load_model('music21_model_1.h5')

In [33]:
pred_note_length = 100

int_to_note = dict((n, note) for n, note in enumerate(pitchnames))

start_point = np.random.randint(0, len(input_seq) - 1)

pattern = input_seq[start_point]
output_pred = []

for i in range(pred_note_length):
    input_pred = np.reshape(pattern, (1, len(pattern), 1))
    result = model_1.predict(input_pred, verbose = 0)
    index = np.argmax(result)
    output_pred.append(int_to_note[index])
    pattern = np.append(pattern, index/n_vocab)
    pattern = pattern[1:len(pattern)]

In [34]:
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in output_pred:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [35]:
midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='test_music21_output_1.mid')

'test_music21_output_1.mid'

## Using 2-layer stacked LSTM

In [36]:
model_2 = tf.keras.Sequential([
    tf.keras.layers.LSTM(256, input_shape = (input_seq.shape[1], input_seq.shape[2]), return_sequences=True, recurrent_dropout = 0.3,),
    tf.keras.layers.LSTM(256, recurrent_dropout = 0.2),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Dense(n_vocab, activation  = 'softmax')
])
model_2.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [37]:
filepath_2="C:/Users/singh/Downloads/Music Generation/Checkpoints/Music21/weights_2.best.hdf5"

checkpoint = ModelCheckpoint(filepath_2,
                             monitor='loss', 
                             verbose=0, 
                             save_best_only=True, 
                             mode='min')
callbacks_list = [checkpoint]

model_2.fit(input_seq, output_seq, epochs=200,  batch_size=128, callbacks = callbacks_list)

Train on 582 samples
Epoch 1/200
582/582 [==============================] - 11s 20ms/sample - loss: 4.0695
Epoch 2/200
582/582 [==============================] - 8s 13ms/sample - loss: 3.7048
Epoch 3/200
582/582 [==============================] - 7s 12ms/sample - loss: 3.6989
Epoch 4/200
582/582 [==============================] - 8s 13ms/sample - loss: 3.5412
Epoch 5/200
582/582 [==============================] - 8s 14ms/sample - loss: 3.4820
Epoch 6/200
582/582 [==============================] - 9s 15ms/sample - loss: 3.4517
Epoch 7/200
582/582 [==============================] - 9s 15ms/sample - loss: 3.4378
Epoch 8/200
582/582 [==============================] - 9s 16ms/sample - loss: 3.4373
Epoch 9/200
582/582 [==============================] - 9s 16ms/sample - loss: 3.4355
Epoch 10/200
582/582 [==============================] - 9s 16ms/sample - loss: 3.4326
Epoch 11/200
582/582 [==============================] - 10s 16ms/sample - loss: 3.4321
Epoch 12/200
582/582 [==================

582/582 [==============================] - 13s 22ms/sample - loss: 1.5072
Epoch 96/200
582/582 [==============================] - 13s 22ms/sample - loss: 1.4684
Epoch 97/200
582/582 [==============================] - 13s 22ms/sample - loss: 1.4723
Epoch 98/200
582/582 [==============================] - 13s 22ms/sample - loss: 1.4810
Epoch 99/200
582/582 [==============================] - 13s 22ms/sample - loss: 1.6056
Epoch 100/200
582/582 [==============================] - 13s 23ms/sample - loss: 1.5233
Epoch 101/200
582/582 [==============================] - 13s 22ms/sample - loss: 1.4965
Epoch 102/200
582/582 [==============================] - 13s 22ms/sample - loss: 1.4339
Epoch 103/200
582/582 [==============================] - 13s 22ms/sample - loss: 1.4314
Epoch 104/200
582/582 [==============================] - 13s 22ms/sample - loss: 1.3611
Epoch 105/200
582/582 [==============================] - 13s 23ms/sample - loss: 1.3554
Epoch 106/200
582/582 [===========================

In [38]:
model_2.save('music21_model_2.h5')  # creates a HDF5 file 'my_model.h5'
del model_2

model_2 = tf.keras.models.load_model('music21_model_2.h5')

In [39]:
pred_note_length = 100

int_to_note = dict((n, note) for n, note in enumerate(pitchnames))

start_point = np.random.randint(0, len(input_seq) - 1)

pattern = input_seq[start_point]
output_pred = []

for i in range(pred_note_length):
    input_pred = np.reshape(pattern, (1, len(pattern), 1))
    
    result = model_2.predict(input_pred, verbose = 0)
    index = np.argmax(result)
    output_pred.append(int_to_note[index])
    pattern = np.append(pattern, index/n_vocab)
    pattern = pattern[1:len(pattern)]

In [40]:
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in output_pred:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [41]:
midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='test_music21_output_2.mid')

'test_music21_output_2.mid'

## Using 3-layer stacked LSTM

In [23]:
model_3 = tf.keras.Sequential([
    tf.keras.layers.LSTM(256, input_shape = (input_seq.shape[1], input_seq.shape[2]), return_sequences=True, recurrent_dropout = 0.3,),
    tf.keras.layers.LSTM(512, return_sequences=True, recurrent_dropout = 0.3),
    tf.keras.layers.LSTM(256, recurrent_dropout = 0.2),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Dense(n_vocab, activation  = 'softmax')
])
model_3.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [36]:
filepath_3="C:/Users/singh/Downloads/Music Generation/Checkpoints/Music21/weights_3-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

checkpoint = ModelCheckpoint(filepath_3,
                             monitor='loss', 
                             verbose=0, 
                             save_best_only=True, 
                             mode='min')
callbacks_list = [checkpoint]

model_3.fit(input_seq, output_seq, epochs=200,  batch_size=128, callbacks = callbacks_list)

Train on 582 samples
Epoch 1/200
582/582 [==============================] - 20s 35ms/sample - loss: 4.0815
Epoch 2/200
582/582 [==============================] - 19s 32ms/sample - loss: 3.6594
Epoch 3/200
582/582 [==============================] - 20s 34ms/sample - loss: 3.5004
Epoch 4/200
582/582 [==============================] - 21s 36ms/sample - loss: 3.4637
Epoch 5/200
582/582 [==============================] - 21s 37ms/sample - loss: 3.4485
Epoch 6/200
582/582 [==============================] - 23s 39ms/sample - loss: 3.4323
Epoch 7/200
582/582 [==============================] - 23s 40ms/sample - loss: 3.4400
Epoch 8/200
582/582 [==============================] - 24s 41ms/sample - loss: 3.4357
Epoch 9/200
582/582 [==============================] - 25s 43ms/sample - loss: 3.4279
Epoch 10/200
582/582 [==============================] - 26s 44ms/sample - loss: 3.4175
Epoch 11/200
582/582 [==============================] - 26s 45ms/sample - loss: 3.4203
Epoch 12/200
582/582 [=========

582/582 [==============================] - 55s 94ms/sample - loss: 1.0797
Epoch 96/200
582/582 [==============================] - 54s 93ms/sample - loss: 1.1051
Epoch 97/200
582/582 [==============================] - 54s 93ms/sample - loss: 1.0236
Epoch 98/200
582/582 [==============================] - 55s 94ms/sample - loss: 1.0272
Epoch 99/200
582/582 [==============================] - 54s 93ms/sample - loss: 1.0027
Epoch 100/200
582/582 [==============================] - 54s 93ms/sample - loss: 0.9992
Epoch 101/200
582/582 [==============================] - 54s 93ms/sample - loss: 0.9964
Epoch 102/200
582/582 [==============================] - 53s 91ms/sample - loss: 0.9745
Epoch 103/200
582/582 [==============================] - 59s 101ms/sample - loss: 0.9605
Epoch 104/200
582/582 [==============================] - 51s 88ms/sample - loss: 0.9875
Epoch 105/200
582/582 [==============================] - 58s 100ms/sample - loss: 0.9852
Epoch 106/200
582/582 [=========================

In [24]:
#model_3.save('music21_model_3.h5')  # creates a HDF5 file 'my_model.h5'
#del model_3

model_3 = tf.keras.models.load_model('music21_model_3.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [25]:
pred_note_length = 100

int_to_note = dict((n, note) for n, note in enumerate(pitchnames))

start_point = np.random.randint(0, len(input_seq) - 1)
print(start_point)

pattern = input_seq[start_point]
output_pred = []

for i in range(pred_note_length):
    input_pred = np.reshape(pattern, (1, len(pattern), 1))
    
    result = model_3.predict(input_pred, verbose = 0)
    index = np.argmax(result)
    output_pred.append(int_to_note[index])
    pattern = np.append(pattern, index/n_vocab)
    pattern = pattern[1:len(pattern)]

392


In [26]:
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in output_pred:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [27]:
midi_stream = stream.Stream(output_notes)

midi_stream.write('midi', fp='test_music21_output_3.mid')

'test_music21_output_3.mid'